In [ ]:
import numpy as np
import random
from random import shuffle
from autodm.llm import get_llm
from scipy.spatial.distance import minkowski

from pydantic import BaseModel, Field
from typing import Union, Literal, Optional, Any, List, Tuple
from rich import print
import matplotlib.pyplot as plt

In [ ]:
width = 30
height = 30

In [ ]:
grid = np.zeros((width, height), dtype=int)

In [ ]:
def random_edge_points(width, height, n_points=2):
    """
    Generate random points on the edges of a gridworld.

    Parameters:
    - width (int): The width of the gridworld.
    - height (int): The height of the gridworld.
    - n_points (int): The number of random points to generate. Default is 2.

    Returns:
    - list: A list of randomly generated points on the edges of the gridworld.
    """

    left_edge = (0, np.random.randint(0, height-1))
    bottom_edge = (np.random.randint(0, width-1), 0)
    right_edge = (width-1, np.random.randint(0, height-1))
    top_edge = (np.random.randint(0, width-1), height-1)
    l = [left_edge, bottom_edge, right_edge, top_edge]
    shuffle(l)
    return l[:n_points]



In [ ]:
p1, p2 = random_edge_points(width, height)

In [ ]:
p1, p2

In [ ]:
def distance(p1, p2, p=2):
    return minkowski(p1, p2, p=p)

In [ ]:
class PathWalker():
    def __init__(self, grid, p1=None, p2=None, p=2, target_distance_per_step=0.6):
        self.grid = grid
        if p1 is None or p2 is None:
            p1, p2 = random_edge_points(width, height)
        self.p1 = p1
        self.p2 = p2
        self.p = p
        self.path = []
        self.target_distance_per_step = target_distance_per_step

    def _distance(self, x, y):
        return distance((x, y), self.p2, p=self.p)
    
    def walk(self, fill_value=1):
        # Reset the path
        self.path = []
        # Set the start x, y to point 1
        x, y = p1
        self.path.append((x, y))
        current_num_steps = 0
        target_num_steps = np.ceil(distance(p1, p2) / self.target_distance_per_step)
        while(current_distance:=self._distance(x, y) > 1):
            # Let's sample from a binomial distribution to decide whether we should decrease or increaes distance.
            # distance represents the smallest number of steps
            # If the current number of steps is less than the target number of steps, we raise the likelihood of increasing the distance
            # If the current number of steps is greater than the target number of steps, we decrease thelikelihood of increasing the distance
            prob_should_decrease_distance = HELP ME FIGURE THIS OUT
            print(prob_should_decrease_distance)
            should_decase_distance = np.random.binomial(1, prob_should_decrease_distance)
            if should_decase_distance:
                # Decrease distance
                x, y = self._decrease_distance(x, y)
            else:
                # Increase distance
                x, y = self._increase_distance(x, y)
            self.path.append((x, y))
            current_num_steps += 1
        return self.path
        
        
    def _adjacent_cells(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x-1, y))
        if x < width-1:
            adjacent_cells.append((x+1, y))
        if y > 0:
            adjacent_cells.append((x, y-1))
        if x < height-1:
            adjacent_cells.append((x, y+1))
        return adjacent_cells
    
    def _decrease_distance(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = self._adjacent_cells(x, y)
        # Find the cells that decrease the distance to the target
        distances = [self._distance(x, y) for x, y in adjacent_cells]
        idx = np.argmin(distances)
        return adjacent_cells[idx]
    
    def _increase_distance(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = self._adjacent_cells(x, y)
        # Find the cells that increase the distance to the target
        distances = [self._distance(x, y) for x, y in adjacent_cells]
        available_choices = [point for point, distance in zip(adjacent_cells, distances) if distance > self._distance(x, y)]
        idx = np.random.choice(len(available_choices))
        return available_choices[idx]
    
    def add_to_grid(self, path):
        for x, y in path:
            self.grid[x, y] = 1
        return self.grid

In [ ]:
class PathWalker:
    def __init__(self, grid, p1=None, p2=None, p=2, target_distance_per_step=0.6):
        self.grid = grid
        self.width = grid.shape[1]
        self.height = grid.shape[0]
        if p1 is None or p2 is None:
            p1, p2 = random_edge_points(self.width, self.height)
        self.p1 = p1
        self.p2 = p2
        self.p = p
        self.path = []
        self.target_distance_per_step = target_distance_per_step

    def _distance(self, x, y):
        return distance((x, y), self.p2, p=self.p)
    
    def walk(self, fill_value=1):
        # Reset the path
        self.path = []
        # Set the start x, y to point 1
        x, y = self.p1
        self.path.append((x, y))
        current_num_steps = 0
        target_num_steps = np.ceil(distance(self.p1, self.p2, p=self.p) / self.target_distance_per_step)
        while (current_distance := self._distance(x, y)) > 1:
            # Calculate probability to decrease distance
            prob_should_decrease_distance = max(0, min(1, (target_num_steps - current_num_steps) / target_num_steps))
            print(f"Probability to decrease distance: {prob_should_decrease_distance}")
            should_decrease_distance = np.random.binomial(1, prob_should_decrease_distance)
            if should_decrease_distance:
                # Decrease distance
                x, y = self._decrease_distance(x, y)
            else:
                # Increase distance
                x, y = self._increase_distance(x, y)
            self.path.append((x, y))
            current_num_steps += 1
        return self.path
        
    def _adjacent_cells(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x-1, y))
        if x < self.width-1:
            adjacent_cells.append((x+1, y))
        if y > 0:
            adjacent_cells.append((x, y-1))
        if y < self.height-1:
            adjacent_cells.append((x, y+1))
        return adjacent_cells
    
    def _decrease_distance(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = self._adjacent_cells(x, y)
        # Find the cells that decrease the distance to the target
        distances = [self._distance(cx, cy) for cx, cy in adjacent_cells]
        idx = np.argmin(distances)
        return adjacent_cells[idx]
    
    def _increase_distance(self, x, y):
        # Find the adjacent cells to the current cell
        adjacent_cells = self._adjacent_cells(x, y)
        # Find the cells that increase the distance to the target
        distances = [self._distance(cx, cy) for cx, cy in adjacent_cells]
        available_choices = [point for point, distance in zip(adjacent_cells, distances) if distance > self._distance(x, y)]
        if available_choices:
            idx = np.random.choice(len(available_choices))
            return available_choices[idx]
        else:
            return random.choice(adjacent_cells)  # Fallback to any adjacent cell if no increase is possible
    
    def add_to_grid(self, path):
        for x, y in path:
            self.grid[x, y] = 1
        return self.grid

In [ ]:
walker = PathWalker(grid)

In [ ]:
walker.path

In [ ]:
points = walker.walk()

In [ ]:
walker.grid

In [ ]:
from llama_index.core.program import LLMTextCompletionProgram

In [ ]:
llm = get_llm()

In [ ]:
scales = Literal['miles', 'feet']

In [ ]:
scales = Literal['miles', 'feet']

class GridWorld(BaseModel):
    width:int = Field(50, description="Width of the grid", ge=1)
    height:int = Field(50, description="Height of the grid", ge=1)
    scale:int = Field(1, description="Number of units ", ge=1)
    scale_unit: scales = Field('miles', description="Unit of the scale")
    grid: Optional[Any] = Field(None, description="Grid of the world")

    def model_post_init(self, __context: Any) -> None:
        self.grid = np.zeros((self.height, self.width), dtype=int)

    def show(self):
        plt.imshow(self.grid, cmap='hot')

    def __str__(self):
        s = ""
        for i in world.grid:
            s += str(i.tolist())
            s += '\n'
        return s
    def __repr__(self):
        return self.__str__()

In [ ]:
REGIONAL_GRID_VALUES = {
    0: "wilderness",
    1: "city/town",
    2: "road",
    3: "water",
    4: "foothills",
    5: "mountain",
}

class MapItem(BaseModel):
    xs: List[int]
    ys: List[int]
    value: int = Field(0, description="Value to be added to the grid")

    def add_to_map(self, grid: GridWorld):
        grid.grid[self.ys, self.xs] = self.value

In [ ]:
world = GridWorld()

In [ ]:
region_s = "\n".join([f"{k}: {v}" for k, v in REGIONAL_GRID_VALUES.items()])
prompt = f"""\
You are a creative dungeon master filling in a D&D map with one of the following categories:
{region_s}
Here is the current grid:
{str(world)}
Add one river or other body of water to the world. \
If it's a river, it should touch at least 2 sides of the map. \
All bodies of water should be connected. \
Please follow the requested json format.

Answer: \
"""

In [ ]:
region_s = "\n".join([f"{k}: {v}" for k, v in REGIONAL_GRID_VALUES.items()])
prompt = f"""\
You are a creative dungeon master filling in a D&D map with one of the following categories:
{region_s}
Here is the current {world.width} x {world.height} grid:
{str(world)}
Please add the following to the map:
2-3 cities/towns
Meandering roads connecting the cities
A river that touches at least 2 sides of the map
Mountains and foothills

Answer: \
"""

In [ ]:
output = get_llm().complete(prompt)

In [ ]:
output

In [ ]:
program = LLMTextCompletionProgram.from_defaults(output_cls=MapItem, prompt_template_str=prompt, llm=get_llm())

In [ ]:
output

In [ ]:
output.add_to_map(world)

In [ ]:
world.show()